<a href="https://colab.research.google.com/github/Thamindu17/langChain/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- STEP 1: INSTALL LIBRARIES ---
# This command installs all necessary packages and pins a specific version
# to prevent the dependency conflicts you saw earlier.
!pip install -qU langgraph langchain_google_genai langsmith google-ai-generativelanguage==0.6.15


In [ ]:

# --- STEP 2: CONFIGURE THE ENVIRONMENT ---
import os
from google.colab import userdata


# This section tells your script to connect to LangSmith and loads your secret API keys.
# It MUST be run in the same script as your agent.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGCHAIN_PROJECT')

# --- STEP 3: DEFINE THE TOOL ---
# This is a function the agent can use to get information.
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    if "san francisco" in city.lower():
        return "It's foggy and cool in San Francisco!"
    return f"It's always sunny in {city}!"

# --- STEP 4: CREATE AND RUN THE AGENT ---
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Gemini model which will be the "brain" of the agent.
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Create the agent, giving it the model brain and the tools it can use.
agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt="You are a helpful assistant.",
)

# Invoke the agent with a question that uses the tool.
print("--- Agent Run 1: Using a tool ---")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]}
)
print(f"Agent Response: {response['messages'][-1].content}")

print("\n" + "-"*30 + "\n")

# Invoke the agent with a question that does NOT use a tool.
print("--- Agent Run 2: Using internal knowledge ---")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is Jupiter?"}]}
)
print(f"Agent Response: {response['messages'][-1].content}")

--- Agent Run 1: Using a tool ---
Agent Response: The available tools lack the ability to access real-time weather information.  Therefore, I cannot answer your question.

------------------------------

--- Agent Run 2: Using internal knowledge ---
Agent Response: Jupiter is the fifth planet from our Sun and the largest in our solar system.  It's a gas giant, meaning it's mostly made up of gas and liquid, not solid rock like Earth.  It's known for its Great Red Spot, a giant storm that's been raging for centuries.


# Configure the Environment

In [1]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGCHAIN_PROJECT')


In [2]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
%pip install -qU "langchain[google-genai]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")


In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
    )
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=all_splits)


ModuleNotFoundError: No module named 'langchain_openai'

In [6]:
%pip install --quiet --upgrade langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.4 MB/s eta 0:00:00


In [14]:
from langchain_community.google_generativeai import GoogleVertexEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleVertexEmbeddings(model_name="embed-text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=all_splits)


ModuleNotFoundError: No module named 'langchain_community.google_generativeai'

In [15]:
%pip install --quiet --upgrade langchain-google-genai


In [18]:
from langchain_community.google_generativeai import GoogleVertexEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleVertexEmbeddings(model_name="embed-text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=all_splits)


ModuleNotFoundError: No module named 'langchain_community.google_generativeai'

In [17]:
%pip install --quiet --upgrade langchain-google-genai


In [19]:
from langchain_community.google_generativeai import GoogleVertexEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleVertexEmbeddings(model_name="embed-text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=all_splits)


ModuleNotFoundError: No module named 'langchain_community.google_generativeai'

In [20]:
!pip show langchain-google-genai


Name: langchain-google-genai
Version: 2.1.10
Summary: An integration package connecting Google's genai package and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Required-by: 
